In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
! git clone https://github.com/cvg/LightGlue.git

Cloning into 'LightGlue'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 386 (delta 144), reused 80 (delta 80), pack-reused 184 (from 2)
Receiving objects: 100% (386/386), 17.43 MiB | 30.45 MiB/s, done.
Resolving deltas: 100% (236/236), done.


In [ ]:
cd LightGlue

/content/LightGlue


In [ ]:
! python -m pip install -e .

Obtaining file:///content/LightGlue
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm.notebook import tqdm
from PIL import Image
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 933120000
from lightglue import LightGlue, SuperPoint, DISK, SIFT, ALIKED, DoGHardNet
from lightglue.utils import load_image, rbd
plt.rcParams['figure.figsize'] = [15, 15]

/content/LightGlue/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [ ]:
cd ..

/content


In [ ]:
# ---------------------SUPERPOINT----------------------------
#  Optimized
# extractor = SuperPoint(max_num_keypoints=2048).eval().cuda()  # load the extractor
# matcher = LightGlue(features='superpoint').eval().cuda()  # load the matcher

# Full accuracy
# extractor = SuperPoint(max_num_keypoints=None).cuda()
# matcher = LightGlue(features='superpoint', depth_confidence=-1, width_confidence=-1).cuda()

# Full speed
# extractor = SuperPoint(max_num_keypoints=1024).cuda()
# matcher = LightGlue(features='superpoint', depth_confidence=0.9, width_confidence=0.95).cuda()

# ---------------------DISK----------------------------
#  Optimized
# extractor = DISK(max_num_keypoints=2048).eval().cuda()
# matcher = LightGlue(features='disk').cuda()

# Full accuracy
# extractor = DISK(max_num_keypoints=None).eval().cuda()
# matcher = LightGlue(features='disk', depth_confidence=-1, width_confidence=-1).cuda()

# Full speed
# extractor = DISK(max_num_keypoints=1024).eval().cuda()
# matcher = LightGlue(features='disk', depth_confidence=0.9, width_confidence=0.95).cuda()

# ---------------------SIFT----------------------------
#  Optimized
# extractor = SIFT(max_num_keypoints=2048).eval().cuda()
# matcher = LightGlue(features='sift').cuda()

# Full accuracy
# extractor = SIFT(max_num_keypoints=None).eval().cuda()
# matcher = LightGlue(features='sift', depth_confidence=-1, width_confidence=-1).cuda()

# Full speed
# extractor = SIFT(max_num_keypoints=1024).eval().cuda()
# matcher = LightGlue(features='sift', depth_confidence=0.9, width_confidence=0.95).cuda()

# ---------------------ALIKED----------------------------
#  Optimized
# extractor = ALIKED(max_num_keypoints=2048).eval().cuda()
# matcher = LightGlue(features='aliked').cuda()

# Full accuracy
# extractor = ALIKED(max_num_keypoints=None).eval().cuda()
# matcher = LightGlue(features='aliked', depth_confidence=-1, width_confidence=-1).cuda()

# Full speed
# extractor = ALIKED(max_num_keypoints=1024).eval().cuda()
# matcher = LightGlue(features='aliked', depth_confidence=0.9, width_confidence=0.95).cuda()


Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/sift_lightglue.pth" to /root/.cache/torch/hub/checkpoints/sift_lightglue_v0-1_arxiv.pth
100%|██████████| 45.4M/45.4M [00:00<00:00, 184MB/s]


In [ ]:




# Read image, convert to gray, and resize based on resize factor
def read_image(path, resize=1.0):
    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"Image not found at path: {path}")

    # Resize image if resize factor is less than 1
    if 0 < resize < 1.0:
        h, w = img.shape[:2]
        new_size = (int(w * resize), int(h * resize))
        img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img_gray, img, img_rgb

In [ ]:
import random
def homography(pairs):
    rows = []
    for i in range(pairs.shape[0]):
        p1 = np.append(pairs[i][0:2], 1)
        p2 = np.append(pairs[i][2:4], 1)
        row1 = [0, 0, 0, p1[0], p1[1], p1[2], -p2[1]*p1[0], -p2[1]*p1[1], -p2[1]*p1[2]]
        row2 = [p1[0], p1[1], p1[2], 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1], -p2[0]*p1[2]]
        rows.append(row1)
        rows.append(row2)
    rows = np.array(rows)
    U, s, V = np.linalg.svd(rows)
    H = V[-1].reshape(3, 3)
    H = H/H[2, 2] # standardize to let w*H[2,2] = 1
    return H

def random_point(matches, k=4):
    idx = random.sample(range(len(matches)), k)
    point = [matches[i] for i in idx ]
    return np.array(point)

In [ ]:
def random_point(matches, k=4):
    idx = random.sample(range(len(matches)), k)
    point = [matches[i] for i in idx ]
    return np.array(point)

In [ ]:
def get_error(points, H):
    num_points = len(points)
    all_p1 = np.concatenate((points[:, 0:2], np.ones((num_points, 1))), axis=1)
    all_p2 = points[:, 2:4]
    estimate_p2 = np.zeros((num_points, 2))
    for i in range(num_points):
        temp = np.dot(H, all_p1[i])
        estimate_p2[i] = (temp/temp[2])[0:2] # set index 2 to 1 and slice the index 0, 1
    # Compute error
    errors = np.linalg.norm(all_p2 - estimate_p2 , axis=1) ** 2

    return errors

In [ ]:
def ransac(matches, threshold, iters):
    num_best_inliers = 0

    for i in range(iters):
        points = random_point(matches)
        H = homography(points)

        #  avoid dividing by zero
        if np.linalg.matrix_rank(H) < 3:
            continue

        errors = get_error(matches, H)
        idx = np.where(errors < threshold)[0]
        inliers = matches[idx]

        num_inliers = len(inliers)
        if num_inliers > num_best_inliers:
            best_inliers = inliers.copy()
            num_best_inliers = num_inliers
            best_H = H.copy()

    print("inliers/matches: {}/{}".format(num_best_inliers, len(matches)))
    return best_inliers, best_H

In [ ]:
def stitch_img(left, right, H):
    # print("Stitching image ...")

    # Normalize images to float in [0,1]
    left = cv2.normalize(left.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    right = cv2.normalize(right.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # Get dimensions of both images
    height_l, width_l, _ = left.shape
    height_r, width_r, _ = right.shape

    # Compute corners for the left image and transform them using H
    corners_left = np.array([[0, 0, 1],
                             [width_l, 0, 1],
                             [width_l, height_l, 1],
                             [0, height_l, 1]]).T  # shape (3,4)
    warped_corners_left = H @ corners_left
    warped_corners_left /= warped_corners_left[2, :]  # Normalize homogeneous coordinates

    # Compute corners for the right image (identity transform)
    corners_right = np.array([[0, 0, 1],
                              [width_r, 0, 1],
                              [width_r, height_r, 1],
                              [0, height_r, 1]]).T  # shape (3,4)

    # Combine corners to find overall bounds
    all_x = np.concatenate((warped_corners_left[0, :], corners_right[0, :]))
    all_y = np.concatenate((warped_corners_left[1, :], corners_right[1, :]))

    min_x, max_x = np.min(all_x), np.max(all_x)
    min_y, max_y = np.min(all_y), np.max(all_y)

    # Create a translation matrix to shift all images so that no coordinate is negative
    tx = -min_x if min_x < 0 else 0
    ty = -min_y if min_y < 0 else 0
    translation_mat = np.array([[1, 0, tx],
                                [0, 1, ty],
                                [0, 0, 1]])

    # New canvas size: use ceiling to ensure full coverage
    width_new = int(np.ceil(max_x - min_x))
    height_new = int(np.ceil(max_y - min_y))
    size = (width_new, height_new)

    # Warp left image with the composite transform: translation_mat @ H
    warped_left = cv2.warpPerspective(left, translation_mat @ H, size)
    # Warp right image with just the translation matrix (identity warp + translation)
    warped_right = cv2.warpPerspective(right, translation_mat, size)

    # Vectorized blending:
    # Create masks where any channel is non-zero (assumed as non-black)
    mask_left = np.any(warped_left != 0, axis=2)
    mask_right = np.any(warped_right != 0, axis=2)

    # Initialize the stitched image as black
    stitch_image = np.zeros_like(warped_left)

    # Pixels only from left image
    only_left = mask_left & ~mask_right
    stitch_image[only_left] = warped_left[only_left]

    # Pixels only from right image
    only_right = mask_right & ~mask_left
    stitch_image[only_right] = warped_right[only_right]

    # Pixels where both images contribute (average the pixel values)
    both = mask_left & mask_right
    stitch_image[both] = (warped_left[both] + warped_right[both]) / 2

    return stitch_image

In [ ]:
import cv2
import numpy as np
import torch
import math

# if you haven’t already got a counter somewhere:
kct = -50

def torch_to_cv2_image(tensor: torch.Tensor) -> np.ndarray:
    """
    Convert a C×H×W RGB torch tensor (on CPU or CUDA) to a H×W×C BGR uint8 numpy array.
    """
    if tensor.is_cuda:
        tensor = tensor.cpu()
    image = tensor.clone().detach()
    # (C,H,W) -> (H,W,C)
    image = image.permute(1, 2, 0).numpy()
    # assume floats in [0,1] or ints in [0,255]
    if image.dtype != np.uint8:
        image = np.clip(image * 255, 0, 255).astype(np.uint8)
    # RGB->BGR
    return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

def keep_top_left_percent(image, k: float):
    """
    Keep only the top and left k-percent of pixels (an upside-down 'L'), black out the rest.
    Works on either
      • numpy.ndarray (H×W×C BGR)  or
      • torch.Tensor   (C×H×W RGB floats [0,1] or [0,255], on CPU or CUDA)
    """
    global kct
    # --- validate k ---
    if not (0 < k <= 1):
        raise ValueError("Parameter k must be a float between 0 and 1.")

    # --- branch on type ---
    if isinstance(image, torch.Tensor):
        print("Is tensor")
        # tensor path: C×H×W
        C, H, W = image.shape
        top_h = math.ceil(k * H)
        left_w = math.ceil(k * W)

        # build mask on same device & dtype
        mask2d = torch.zeros((H, W), dtype=image.dtype, device=image.device)
        mask2d[:top_h, :] = 1
        mask2d[:, :left_w] = 1

        # expand to C×H×W
        mask3d = mask2d.unsqueeze(0).expand(C, H, W)

        # apply mask
        out_t = image * mask3d

        # save out as JPEG
        cv2_img = torch_to_cv2_image(out_t)
        kct += 1
        cv2.imwrite(f"{kct}.jpg", cv2_img)

        return out_t

    elif isinstance(image, np.ndarray):
        print("Is not tensor!")
        # numpy path: H×W×C BGR
        H, W = image.shape[:2]
        top_h = math.ceil(k * H)
        left_w = math.ceil(k * W)

        mask = np.zeros((H, W), dtype=bool)
        mask[:top_h, :] = True
        mask[:, :left_w] = True

        black = np.zeros_like(image)
        out_np = np.where(mask[..., None], image, black)

        kct += 1
        cv2.imwrite(f"{kct}.jpg", out_np)

        return out_np

    else:
        raise TypeError(
            f"Unsupported image type {type(image)} – must be numpy.ndarray or torch.Tensor."
        )


def resize_torch(image0, new_size):
  # image0 is a C×H×W float tensor on CUDA
  # 1) send to CPU and turn into a H×W×C NumPy array
  np_img = image0.cpu().permute(1, 2, 0).numpy()

  # 2) do your OpenCV resize
  h, w = np_img.shape[:2]
  new_w = (w * int(new_size * 10)) // 10
  new_h = (h * int(new_size * 10)) // 10
  resized_np = cv2.resize(np_img, (new_w, new_h))

  # 3) (optional) turn back into a tensor, same dtype/device as before
  return torch.from_numpy(resized_np).permute(2, 0, 1).to(image0.dtype).cuda()


def crop_tensor_image(image: torch.Tensor, h1: int, w1: int, h2: int, w2: int) -> torch.Tensor:
    """
    Crop an image tensor based on bounding box coordinates.

    Args:
        image (torch.Tensor): Image tensor of shape (C, H, W), values in [0,1] or [0,255].
        h1 (int): Top coordinate (inclusive).
        w1 (int): Left coordinate (inclusive).
        h2 (int): Bottom coordinate (exclusive).
        w2 (int): Right coordinate (exclusive).

    Returns:
        torch.Tensor: Cropped image tensor of shape (C, h2-h1, w2-w1).
    """
    if image.ndim != 3:
        raise ValueError("Input image must be a 3D tensor (C, H, W)")

    C, H, W = image.shape

    # Clamp coordinates to image boundaries
    h1 = max(0, min(H, h1))
    h2 = max(0, min(H, h2))
    w1 = max(0, min(W, w1))
    w2 = max(0, min(W, w2))

    if h1 >= h2 or w1 >= w2:
        raise ValueError("Invalid bounding box: (h1, w1) must be above and to the left of (h2, w2)")

    return image[:, h1:h2, w1:w2]

In [ ]:
def plot_matches(matches, total_img, pos1=(0,0), pos2=(0,0)):
    """
    matches: Nx4 array where each row is [x1, y1, x2, y2]
    total_img: composite image with both images pasted on it.
    pos1: (x, y) top-left position of image1 in total_img.
    pos2: (x, y) top-left position of image2 in total_img.
    """
    fig, ax = plt.subplots()
    ax.imshow(total_img.astype(np.uint8))
    ax.set_aspect('equal')

    # Draw keypoints for image1 and image2 (adjusted by their positions)
    pts1 = matches[:, :2] + np.array(pos1)
    pts2 = matches[:, 2:] + np.array(pos2)
    ax.plot(pts1[:, 0], pts1[:, 1], 'xr', markersize=5)
    ax.plot(pts2[:, 0], pts2[:, 1], 'xr', markersize=5)

    # Draw lines connecting the matching points
    for (x1, y1), (x2, y2) in zip(pts1, pts2):
        ax.plot([x1, x2], [y1, y2], 'r', linewidth=0.5)

    plt.title("Feature Matches")
    plt.show()

In [ ]:
import os

def convert_image_number_base(filepath: str, b1: int, b2: int) -> str:
    if not (1 <= b1 <= 10 and 1 <= b2 <= 10):
        raise ValueError("Bases b1 and b2 must be between 1 and 10 (inclusive)")

    # Split into directory, filename, extension
    dirname, filename = os.path.split(filepath)
    name, ext = os.path.splitext(filename)

    if ext.lower() not in ['.jpg', '.png']:
        raise ValueError("File must be a .jpg or .png")

    # Ensure the number part is 4 digits
    if not (len(name) == 4 and name.isdigit()):
        raise ValueError("Filename must contain a 4-digit number")

    # Convert number from base b1 to int
    try:
        number = int(name, base=b1)
    except ValueError:
        raise ValueError(f"The number '{name}' is not valid in base {b1}")

    # Convert from int to base b2
    def to_base(n: int, base: int) -> str:
        if n == 0:
            return '0'
        digits = []
        while n:
            digits.append(str(n % base))
            n //= base
        return ''.join(reversed(digits))

    new_number = to_base(number, b2).zfill(4)
    new_filename = new_number + ext
    return os.path.join(dirname, new_filename) if dirname else new_filename

print(convert_image_number_base('0005.jpg', 10, 5))         # '0010.jpg'
print(convert_image_number_base('image/0006.png', 10, 5))   # 'image/0011.png'

0010.jpg
image/0011.png


In [ ]:
def poz(matrix_size, i):
  init_i = i

  if i < 1:
    return None
  if i > (matrix_size * matrix_size) - 1:
    return None

  max_line = matrix_size - 1

  diag_nr = 1
  x = diag_nr
  y = 0
  prev_poz = 0, 0
  while True:
    i -= 1
    if i == 0:
      if init_i < ((matrix_size * (matrix_size + 1)) // 2):
        prev_x, prev_y = prev_poz
        if prev_x == 0:
          return x, y, prev_x + prev_y + 1, prev_x + prev_y + 1
        else:
          return x, y, prev_x + prev_y + 1, prev_x + prev_y
      return x, y, matrix_size, matrix_size

    if i == 1:
      prev_poz = x, y

    if x == 0 or y == max_line:
      diag_nr += 1

      if diag_nr > max_line:
        x = max_line
        y = diag_nr - max_line
      else:
        x = diag_nr
        y = 0
    else:
      x -= 1
      y += 1

def poz_list(path, paths):
  max_0, max_1 = 0, 0
  for path1 in paths:
    if path == path1:
      break
    # path1 = convert_image_number_base(path1, 10, 5)
    x, y = int(path1.split(".jpg")[0][2:][0]), int(path1.split(".jpg")[0][2:][1])
    max_0 = max(max_0, x)
    max_1 = max(max_1, y)
  return int(path.split(".jpg")[0][2:][0]), int(path.split(".jpg")[0][2:][1]), max_0 + 1, max_1 + 1

import torchvision.utils as vutils

def save_image(tensor: torch.Tensor, filename: str = "image.png"):
    """
    Save a tensor as an image file.

    Args:
        tensor (torch.Tensor): Image tensor of shape (C, H, W).
        filename (str): Path to save the image.
    """
    # If the tensor is [0, 255] floats, normalize it to [0, 1]
    if tensor.max() > 1.0:
        tensor = tensor / 255.0

    vutils.save_image(tensor, filename)

In [ ]:
from PIL import Image
import numpy as np

def detect_black_border_depths(image_pil, black_threshold=10):
    """
    Detects how much black padding exists on each edge of the image separately.

    Args:
        image_pil: PIL.Image object (RGB or grayscale).
        black_threshold: Maximum pixel value considered "black" (0-255).

    Returns:
        (top, bottom, left, right): number of black pixels to crop from each side.
    """
    image_np = np.array(image_pil)

    if image_np.ndim == 3:
        # Convert to grayscale if RGB
        image_np = np.mean(image_np, axis=2)

    h, w = image_np.shape

    # Initialize cropping values
    top = 0
    bottom = 0
    left = 0
    right = 0

    # Detect top
    for y in range(h):
        if np.all(image_np[y, :] <= black_threshold):
            top += 1
        else:
            break

    # Detect bottom
    for y in range(h-1, -1, -1):
        if np.all(image_np[y, :] <= black_threshold):
            bottom += 1
        else:
            break

    # Detect left
    for x in range(w):
        if np.all(image_np[:, x] <= black_threshold):
            left += 1
        else:
            break

    # Detect right
    for x in range(w-1, -1, -1):
        if np.all(image_np[:, x] <= black_threshold):
            right += 1
        else:
            break

    return top, bottom, left, right

def crop_black_borders(image_pil, black_threshold=10):
    """
    Crop black borders independently from each edge.

    Args:
        image_pil: PIL.Image object (RGB or grayscale).
        black_threshold: Maximum pixel value considered "black" (0-255).

    Returns:
        Cropped PIL.Image object.
    """
    top, bottom, left, right = detect_black_border_depths(image_pil, black_threshold)
    w, h = image_pil.size

    # Calculate new box
    left_crop = left
    upper_crop = top
    right_crop = w - right
    lower_crop = h - bottom

    if left_crop >= right_crop or upper_crop >= lower_crop:
        # If everything is cropped away, return original (or could raise an error)
        print("Warning: Cropping would remove entire image. Returning original.")
        return image_pil

    return image_pil.crop((left_crop, upper_crop, right_crop, lower_crop))

In [ ]:
def torch_image_shape(image):
  np_img = image.cpu().permute(1, 2, 0).numpy()
  h, w = np_img.shape[:2]
  return h, w

def LightGlue_diagonal_matching(
    img_path1,
    img_path2,
    basepath_img2,
    path_list,
    i_value,
    matrix_size,
    output_dir=f"./lightglue_{kct}_output",
    ransac_thresholding=100,
    percentage_of_image_used=1.0,

):
  left_gray, left_origin, left_rgb = read_image(img_path1, 0.7)
  right_gray, right_origin, right_rgb = read_image(img_path2, 0.7)

  # right_height, right_width = img_path2.shape[:2]
  # new_height = int(left_height * percentage_of_image_used)
  # new_width = int(left_width * percentage_of_image_used)

  image0 = load_image(img_path1).cuda()
  image1 = load_image(img_path2).cuda()

  image0 = resize_torch(image0)
  image1 = resize_torch(image1)

  # image0 = keep_top_left_percent(image0, percentage_of_image_used)
  image1 = keep_top_left_percent(image1, percentage_of_image_used)

  poz_h, poz_w, nr_h, nr_w = poz_list(basepath_img2, path_list)

  # poz_h, poz_w, nr_h, nr_w = poz(matrix_size, i_value)

  print(f"We want to crop the matrix crop {(poz_h, poz_w)} out of a {matrix_size - 1, matrix_size - 1} matrix!")

  h, w = torch_image_shape(image0)

  print(f"Image size of {h, w}")

  h_size, w_size = h // nr_h, w // nr_w

  print(f"Tile size of {(h_size, w_size)}")

  matches = []

  factor = 0

  while len(matches) < 10:

    top_left_h, top_left_w = int(h_size * max(0, poz_h - 0.75 - factor)), int(w_size * max(0, poz_w - 0.75 - factor))

    bottom_right_h, bottom_right_w = int(min(h, h_size * (poz_h + 1.5 + factor))), int(min(w, w_size * (poz_w + 1.5 + factor)))

    print(f"Box proposed is {(top_left_h, top_left_w, bottom_right_h, bottom_right_w)}")

    if factor != 0:
      image0 = load_image(img_path1).cuda()
      image0 = resize_torch(image0)
    image0 = crop_tensor_image(image0, top_left_h, top_left_w, bottom_right_h, bottom_right_w)

    # cropped = crop_tensor_image(image0, top_left_h, top_left_w, bottom_right_h, bottom_right_w)
    save_image(image0, f"00000_{i_value}_{factor}.png")
    save_image(image1, f"00001_{i_value}_{factor}.png")

    # extractor = SuperPoint(max_num_keypoints=2048).eval().cuda()  # load the extractor
    # matcher = LightGlue(features='superpoint').eval().cuda()  # load the matcher
    # extractor = SuperPoint(max_num_keypoints=1024).cuda()
    # matcher = LightGlue(features='superpoint', depth_confidence=0.9, width_confidence=0.95).cuda()
    extractor = SuperPoint(max_num_keypoints=None).cuda()
    matcher = LightGlue(features='superpoint', depth_confidence=-1, width_confidence=-1).cuda()

    # extractor = DISK(max_num_keypoints=2048).eval().cuda()
    # matcher = LightGlue(features='disk', depth_confidence=0.9, width_confidence=0.95).cuda()

    # extractor = SIFT(max_num_keypoints=None).eval().cuda()
    # matcher = LightGlue(features='sift', depth_confidence=-1, width_confidence=-1).cuda()
    # extractor = SIFT(max_num_keypoints=1024).eval().cuda()
    # matcher = LightGlue(features='sift', depth_confidence=0.9, width_confidence=0.95).cuda()

    feats0 = extractor.extract(image0)
    feats1 = extractor.extract(image1)

    matches01 = matcher({'image0': feats0, 'image1': feats1})
    feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]
    matches = matches01['matches']
    points0 = feats0['keypoints'][matches[..., 0]]
    points1 = feats1['keypoints'][matches[..., 1]]

    matches = np.concatenate((points0.cpu(), points1.cpu()), axis=1)

    print(f"\n\n\nAvem {len(matches)} matches!\n\n\n")

    for i in range(len(matches)):
      matches[i][0] += top_left_w
      matches[i][1] += top_left_h

    factor = factor + 0.25

    if factor > 2:
      break
  # height = left_rgb.shape[0] + right_rgb.shape[0]
  # width = left_rgb.shape[1] + right_rgb.shape[1]
  # total_img = np.zeros((height, width, 3), dtype=np.uint8)
  # pos1 = (0, 0)  # top-left corner for left image
  # pos2 = (left_rgb.shape[1], left_rgb.shape[0])  # place right image diagonally

  # # Paste the images onto the canvas
  # total_img[pos1[1]:pos1[1]+left_rgb.shape[0], pos1[0]:pos1[0]+left_rgb.shape[1]] = left_rgb
  # total_img[pos2[1]:pos2[1]+right_rgb.shape[0], pos2[0]:pos2[0]+right_rgb.shape[1]] = right_rgb

  # # Plot the matches on the composite image using the corresponding positions.
  # plot_matches(matches, total_img, pos1=pos1, pos2=pos2)

  inliers, H = ransac(matches, 100, 2000)

  kaze_stitched = stitch_img(left_rgb, right_rgb, H)

  kaze_stitched_uint8 = (kaze_stitched * 255).astype(np.uint8)
  image_pil = Image.fromarray(kaze_stitched_uint8)

  output_filename = f"0000-{os.path.splitext(os.path.basename(img_path2))[0]}_LightGlue_{percentage_of_image_used}.jpg"
  output_filepath = os.path.join(output_dir, output_filename)

  print(f"Saving at {output_filepath}")

  cropped_image = crop_black_borders(image_pil, black_threshold=10)

  cropped_image.save(output_filepath, "JPEG", quality=95)

  # border_depth = detect_black_border_depth(image_pil)

  # print(f"Detected black border depth: {border_depth} pixels")

  # # Optional: crop automatically if needed
  # if border_depth > 0:
  #     w, h = image_pil.size
  #     cropped = image_pil.crop((border_depth, border_depth, w - border_depth, h - border_depth))
  #     cropped.save(output_filepath, "JPEG", quality=95)
  # else:
  #     image_pil.save(output_filepath, "JPEG", quality=95)

  return output_filename

In [ ]:
def torch_image_shape(image):
  np_img = image.cpu().permute(1, 2, 0).numpy()
  h, w = np_img.shape[:2]
  return h, w

def LightGlue_matrix_scan_diagonal_matching(
    img_path1,
    img_path2,
    basepath_img2,
    path_list,
    i_value,
    matrix_size,
    output_dir=f"./lightglue_{kct}_output",
    ransac_thresholding=100,
    percentage_of_image_used=1.0,

):

  poz_h, poz_w, nr_h, nr_w = poz_list(basepath_img2, path_list)

  print(f"We want to crop the matrix crop {(poz_h, poz_w)} out of a {matrix_size - 1, matrix_size - 1} matrix!")
  print(f"{nr_h}, {nr_w}")

  matches_maxx = []

  # for iii in range(0, nr_h * 10 - 5, 5):
  #   for jjj in range(0, nr_w * 10 - 5, 5):
  #     i = iii / 10
  #     j = jjj / 10
  for i in range(nr_h):
    for j in range(nr_w):
      torch.cuda.empty_cache()
      left_gray, left_origin, left_rgb = read_image(img_path1, 0.8)
      right_gray, right_origin, right_rgb = read_image(img_path2, 0.8)

      # right_height, right_width = img_path2.shape[:2]
      # new_height = int(left_height * percentage_of_image_used)
      # new_width = int(left_width * percentage_of_image_used)

      image0 = load_image(img_path1).cuda()
      image1 = load_image(img_path2).cuda()

      image0 = resize_torch(image0, 0.8)
      image1 = resize_torch(image1, 0.8)

      # image0 = keep_top_left_percent(image0, percentage_of_image_used)
      image1 = keep_top_left_percent(image1, percentage_of_image_used)

      h, w = torch_image_shape(image0)

      print(f"Image size of {h, w}")

      h_size, w_size = h // nr_h, w // nr_w

      print(f"Tile size of {(h_size, w_size)}")

      flag = True

      print(f"h_size = {h_size}")
      print(f"w_size = {w_size}")

      print(f"i = {i}")
      print(f"j = {j}")
      top_left_h = int(max(0, (j * h_size) - (h_size * 0.65)))
      top_left_w = int(max(0, (i * w_size) - (w_size * 0.65)))

      bottom_right_h = int(min(h, ((j + 1) * h_size) + (0.65 * h_size)))
      bottom_right_w = int(min(w, ((i + 1) * w_size) + (0.65 * w_size)))

      print(f"Box proposed is {(top_left_h, top_left_w, bottom_right_h, bottom_right_w)}")

      image0 = crop_tensor_image(image0, top_left_h, top_left_w, bottom_right_h, bottom_right_w)

      # cropped = crop_tensor_image(image0, top_left_h, top_left_w, bottom_right_h, bottom_right_w)
      save_image(image0, f"00000_{i_value}_{i}_{j}.png")
      save_image(image1, f"00001_{i_value}.png")

      # extractor = SuperPoint(max_num_keypoints=2048).eval().cuda()  # load the extractor
      # matcher = LightGlue(features='superpoint').eval().cuda()  # load the matcher
      # extractor = SuperPoint(max_num_keypoints=1024).cuda()
      # matcher = LightGlue(features='superpoint', depth_confidence=0.9, width_confidence=0.95).cuda()
      # extractor = SuperPoint(max_num_keypoints=None).cuda()
      # matcher = LightGlue(features='superpoint', depth_confidence=-1, width_confidence=-1).cuda()

      # extractor = DISK(max_num_keypoints=2048).eval().cuda()
      # matcher = LightGlue(features='disk', depth_confidence=0.9, width_confidence=0.95).cuda()

      extractor = SIFT(max_num_keypoints=None).eval().cuda()
      matcher = LightGlue(features='sift', depth_confidence=-1, width_confidence=-1).cuda()

      # extractor = SIFT(max_num_keypoints=1024).eval().cuda()
      # matcher = LightGlue(features='sift', depth_confidence=0.9, width_confidence=0.95).cuda()

      try:
        feats0 = extractor.extract(image0)
        feats1 = extractor.extract(image1)

        matches01 = matcher({'image0': feats0, 'image1': feats1})
        feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]
        matches = matches01['matches']
        points0 = feats0['keypoints'][matches[..., 0]]
        points1 = feats1['keypoints'][matches[..., 1]]

        matches = np.concatenate((points0.cpu(), points1.cpu()), axis=1)
      except Exception as e:
        print(f"While doing Deep Learning matching got exception: {e}")
        matches = []


      print(f"\n\n\nAvem {len(matches)} matches!\n\n\n")

      for ii in range(len(matches)):
        matches[ii][0] += top_left_w
        matches[ii][1] += top_left_h

      if len(matches) > len(matches_maxx):
        print("Updated maxx matches!")
        matches_maxx = matches

      # if len(matches_maxx) > 200:
      #   break
  # height = left_rgb.shape[0] + right_rgb.shape[0]
  # width = left_rgb.shape[1] + right_rgb.shape[1]
  # total_img = np.zeros((height, width, 3), dtype=np.uint8)
  # pos1 = (0, 0)  # top-left corner for left image
  # pos2 = (left_rgb.shape[1], left_rgb.shape[0])  # place right image diagonally

  # # Paste the images onto the canvas
  # total_img[pos1[1]:pos1[1]+left_rgb.shape[0], pos1[0]:pos1[0]+left_rgb.shape[1]] = left_rgb
  # total_img[pos2[1]:pos2[1]+right_rgb.shape[0], pos2[0]:pos2[0]+right_rgb.shape[1]] = right_rgb

  # # Plot the matches on the composite image using the corresponding positions.
  # plot_matches(matches, total_img, pos1=pos1, pos2=pos2)

  inliers, H = ransac(matches_maxx, 100, 2000)

  kaze_stitched = stitch_img(left_rgb, right_rgb, H)

  kaze_stitched_uint8 = (kaze_stitched * 255).astype(np.uint8)
  image_pil = Image.fromarray(kaze_stitched_uint8)

  output_filename = f"0000-{os.path.splitext(os.path.basename(img_path2))[0]}_LightGlue_{percentage_of_image_used}.jpg"
  output_filepath = os.path.join(output_dir, output_filename)

  print(f"Saving at {output_filepath}")

  cropped_image = crop_black_borders(image_pil, black_threshold=10)

  cropped_image.save(output_filepath, "JPEG", quality=95)

  # border_depth = detect_black_border_depth(image_pil)

  # print(f"Detected black border depth: {border_depth} pixels")

  # # Optional: crop automatically if needed
  # if border_depth > 0:
  #     w, h = image_pil.size
  #     cropped = image_pil.crop((border_depth, border_depth, w - border_depth, h - border_depth))
  #     cropped.save(output_filepath, "JPEG", quality=95)
  # else:
  #     image_pil.save(output_filepath, "JPEG", quality=95)

  return output_filename

In [ ]:
def torch_image_shape(image):
  np_img = image.cpu().permute(1, 2, 0).numpy()
  h, w = np_img.shape[:2]
  return h, w

def extract_submatrix(flat_list, shape, x1, y1, x2, y2):
    rows, cols = shape
    # Adjust for 1-based indexing
    x1 -= 1
    y1 -= 1
    x2 -= 1
    y2 -= 1

    # Extract and flatten the submatrix directly from flat list
    result = []
    for i in range(x1, x2 + 1):
        start = i * cols + y1
        end = i * cols + y2 + 1
        result.extend(flat_list[start:end])

    return result


def SuperGlue_merge(
    img_path1,
    img_path2,
    output_dir=f"./lightglue_{kct}_output",
    ransac_thresholding=100,
    percentage_of_image_used=1.0,
):
    left_gray, left_origin, left_rgb = read_image(img_path1, 0.8)
    right_gray, right_origin, right_rgb = read_image(img_path2, 0.8)

    image0 = load_image(img_path1).cuda()
    image1 = load_image(img_path2).cuda()

    image0 = resize_torch(image0)
    image1 = resize_torch(image1)

    # image0 = keep_top_left_percent(image0, percentage_of_image_used)
    image1 = keep_top_left_percent(image1, percentage_of_image_used)

    extractor = SuperPoint(max_num_keypoints=None).cuda()
    matcher = LightGlue(features='superpoint', depth_confidence=-1, width_confidence=-1).cuda()

    feats0 = extractor.extract(image0)
    feats1 = extractor.extract(image1)

    matches01 = matcher({'image0': feats0, 'image1': feats1})
    feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]
    matches = matches01['matches']
    points0 = feats0['keypoints'][matches[..., 0]]
    points1 = feats1['keypoints'][matches[..., 1]]

    matches = np.concatenate((points0.cpu(), points1.cpu()), axis=1)

    print(f"\n\n\nAvem {len(matches)} matches!\n\n\n")

    inliers, H = ransac(matches, 100, 2000)

    kaze_stitched = stitch_img(left_rgb, right_rgb, H)

    kaze_stitched_uint8 = (kaze_stitched * 255).astype(np.uint8)
    image_pil = Image.fromarray(kaze_stitched_uint8)

    output_filename = f"0000-{os.path.splitext(os.path.basename(img_path2))[0]}_LightGlue_{percentage_of_image_used}.jpg"
    output_filepath = os.path.join(output_dir, output_filename)

    print(f"Saving at {output_filepath}")

    cropped_image = crop_black_borders(image_pil, black_threshold=10)

    cropped_image.save(output_filepath, "JPEG", quality=95)

    return output_filepath


def LightGlue_divide_et_impera_matching(
    path_list,
    matrix_shape,
    output_dir=f"./lightglue_{kct}_output",
    ransac_thresholding=100,
    percentage_of_image_used=1.0,

):

  if matrix_shape[0] == 1 and matrix_shape[1] == 1:
    return path_list[0]

  if matrix_shape[0] == 1:
    center_h = matrix_shape[0] // 2

    path_list_top = extract_submatrix(
        path_list,
        matrix_shape,
        1, 1,
        center_h, 1
    )
    path_top = LightGlue_divide_et_impera_matching(
        path_list_top,
         (matrix_shape[0] // 2, 1),
        output_dir,
        ransac_thresholding,
        percentage_of_image_used
    )

    path_list_bottom = extract_submatrix(
        path_list,
        matrix_shape,
        center_h + 1, 1,
        matrix_shape[0], 1
    )
    path_bottom = LightGlue_divide_et_impera_matching(
        path_list_bottom,
         (matrix_shape[0] - (matrix_shape[0] // 2), 1),
        output_dir,
        ransac_thresholding,
        percentage_of_image_used
    )

    return SuperGlue_merge(
        path_top,
        path_bottom,
        output_dir,
        ransac_thresholding,
        percentage_of_image_used
    )

  if matrix_shape[1] == 1:
    center_w = matrix_shape[1] // 2

    path_list_left = extract_submatrix(
        path_list,
        matrix_shape,
        1, 1,
        1, center_w
    )
    path_left = LightGlue_divide_et_impera_matching(
        path_list_left,
         (1, matrix_shape[1] // 2),
        output_dir,
        ransac_thresholding,
        percentage_of_image_used
    )

    path_list_right = extract_submatrix(
        path_list,
        matrix_shape,
        1, center_w + 1,
        1, matrix_shape[1]
    )
    path_right = LightGlue_divide_et_impera_matching(
        path_list_right,
         (1, matrix_shape[1] - (matrix_shape[1] // 2)),
        output_dir,
        ransac_thresholding,
        percentage_of_image_used
    )

    return SuperGlue_merge(
        path_left,
        path_right,
        output_dir,
        ransac_thresholding,
        percentage_of_image_used
    )


  center_h, center_w = matrix_shape[0] // 2, matrix_shape[1] // 2

  path_list_top_left = extract_submatrix(path_list, matrix_shape, 1, 1, center_h, center_w)
  path_top_left = LightGlue_divide_et_impera_matching(path_list_top_left, (matrix_shape[0] // 2, matrix_shape[1] // 2), output_dir, ransac_thresholding, percentage_of_image_used)

  path_list_top_right = extract_submatrix(path_list, matrix_shape, 1, center_w + 1, center_h, matrix_shape[1])
  path_top_right = LightGlue_divide_et_impera_matching(path_list_top_right, (matrix_shape[0] // 2, matrix_shape[1] - (matrix_shape[1] // 2)), output_dir, ransac_thresholding, percentage_of_image_used)

  path_list_bottom_left = extract_submatrix(path_list, matrix_shape, center_h + 1, 1, matrix_shape[0], center_w)
  path_bottom_left = LightGlue_divide_et_impera_matching(path_list_bottom_left, (matrix_shape[0] - (matrix_shape[0] // 2), matrix_shape[1] // 2), output_dir, ransac_thresholding, percentage_of_image_used)

  path_list_bottom_right = extract_submatrix(path_list, matrix_shape, center_h + 1, center_w + 1, matrix_shape[0], matrix_shape[1])
  path_bottom_right = LightGlue_divide_et_impera_matching(path_list_bottom_left, (matrix_shape[0] - (matrix_shape[0] // 2), matrix_shape[1] - (matrix_shape[1] // 2)), output_dir, ransac_thresholding, percentage_of_image_used)

  path_top = SuperGlue_merge(path_top_left, path_top_right, output_dir, ransac_thresholding, percentage_of_image_used)
  path_bottom = SuperGlue_merge(path_bottom_left, path_bottom_right, output_dir, ransac_thresholding, percentage_of_image_used)

  return SuperGlue_merge(path_top, path_bottom, output_dir, ransac_thresholding, percentage_of_image_used)

In [ ]:
def stitch_img(left, right, H):
    # print("Stitching image ...")

    # Normalize images to float in [0,1]
    left = cv2.normalize(left.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    right = cv2.normalize(right.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # Get dimensions of both images
    height_l, width_l, _ = left.shape
    height_r, width_r, _ = right.shape

    # Compute corners for the left image and transform them using H
    corners_left = np.array([[0, 0, 1],
                             [width_l, 0, 1],
                             [width_l, height_l, 1],
                             [0, height_l, 1]]).T  # shape (3,4)
    warped_corners_left = H @ corners_left
    warped_corners_left /= warped_corners_left[2, :]  # Normalize homogeneous coordinates

    # Compute corners for the right image (identity transform)
    corners_right = np.array([[0, 0, 1],
                              [width_r, 0, 1],
                              [width_r, height_r, 1],
                              [0, height_r, 1]]).T  # shape (3,4)

    # Combine corners to find overall bounds
    all_x = np.concatenate((warped_corners_left[0, :], corners_right[0, :]))
    all_y = np.concatenate((warped_corners_left[1, :], corners_right[1, :]))

    min_x, max_x = np.min(all_x), np.max(all_x)
    min_y, max_y = np.min(all_y), np.max(all_y)

    # Create a translation matrix to shift all images so that no coordinate is negative
    tx = -min_x if min_x < 0 else 0
    ty = -min_y if min_y < 0 else 0
    translation_mat = np.array([[1, 0, tx],
                                [0, 1, ty],
                                [0, 0, 1]])

    # New canvas size: use ceiling to ensure full coverage
    width_new = int(np.ceil(max_x - min_x))
    height_new = int(np.ceil(max_y - min_y))
    size = (width_new, height_new)

    # Warp left image with the composite transform: translation_mat @ H
    warped_left = cv2.warpPerspective(left, translation_mat @ H, size)
    # Warp right image with just the translation matrix (identity warp + translation)
    warped_right = cv2.warpPerspective(right, translation_mat, size)

    # Vectorized blending:
    # Create masks where any channel is non-zero (assumed as non-black)
    mask_left = np.any(warped_left != 0, axis=2)
    mask_right = np.any(warped_right != 0, axis=2)

    # Initialize the stitched image as black
    stitch_image = np.zeros_like(warped_left)

    # Pixels only from left image
    only_left = mask_left & ~mask_right
    stitch_image[only_left] = warped_left[only_left]

    # Pixels only from right image
    only_right = mask_right & ~mask_left
    stitch_image[only_right] = warped_right[only_right]

    # Pixels where both images contribute (average the pixel values)
    both = mask_left & mask_right
    stitch_image[both] = (warped_left[both] + warped_right[both]) / 2

    return stitch_image

In [ ]:
cd content

[Errno 2] No such file or directory: 'content'
/content


In [ ]:
import os
from tqdm import tqdm

output_path = "./drive/MyDrive/Disertatie/LightGlue/SIFT/051-04-80"

input_path = "./drive/MyDrive/Disertatie/Patches/051-04-80-original" #026-01-91_original

if not os.path.exists(output_path):
  os.mkdir(output_path)

paths = sorted(os.listdir(input_path))
print(paths)

['0000.jpg', '0001.jpg', '0002.jpg', '0003.jpg', '0004.jpg', '0005.jpg', '0006.jpg', '0007.jpg', '0008.jpg', '0009.jpg', '0010.jpg', '0011.jpg', '0012.jpg', '0013.jpg', '0014.jpg', '0015.jpg', '0016.jpg', '0017.jpg', '0018.jpg', '0019.jpg', '0020.jpg', '0021.jpg', '0022.jpg', '0023.jpg', '0024.jpg', '0025.jpg', '0026.jpg', '0027.jpg', '0028.jpg', '0029.jpg', '0030.jpg', '0031.jpg', '0032.jpg', '0033.jpg', '0034.jpg', '0035.jpg', '0036.jpg', '0037.jpg', '0038.jpg', '0039.jpg', '0040.jpg', '0041.jpg', '0042.jpg', '0043.jpg', '0044.jpg', '0045.jpg', '0046.jpg', '0047.jpg', '0048.jpg', '0049.jpg', '0050.jpg', '0051.jpg', '0052.jpg', '0053.jpg', '0054.jpg', '0055.jpg', '0056.jpg', '0057.jpg', '0058.jpg', '0059.jpg', '0060.jpg', '0061.jpg', '0062.jpg', '0063.jpg', '0064.jpg', '0065.jpg', '0066.jpg', '0067.jpg', '0068.jpg', '0069.jpg', '0070.jpg', '0071.jpg', '0072.jpg', '0073.jpg', '0074.jpg', '0075.jpg', '0076.jpg', '0077.jpg', '0078.jpg', '0079.jpg', '0080.jpg', '0081.jpg', '0082.jpg', '00

In [ ]:
def diagonal_submatrix(k, x, y, image_paths):
    matrix_size = k  # Assumes a 10x10 matrix
    """
    Return the k×k submatrix of a width×width grid stored in `image_paths` (row-major order),
    starting at top-left coordinate (x, y), traversed in diagonal order.

    Parameters:
        image_paths (List[str]): Flat list of image image_paths of length width×width.
        k (int): Size of the submatrix (k rows, k columns).
        x (int): Row index of the submatrix's top-left corner (0-based).
        y (int): Column index of the submatrix's top-left corner (0-based).

    Returns:
        List[str]: The k×k submatrix's elements in diagonal order.
    """
    result = []
    # There are 2k-1 diagonals, indexed by sum d = i+j from 0 to 2(k-1).
    for d in range(2 * k - 1):
        # For each diagonal, i goes from min(d, k-1) down to max(0, d-(k-1)).
        start = min(d, k - 1)
        end = max(0, d - (k - 1))
        for i in range(start, end - 1, -1):
            j = d - i
            global_i = x + i
            global_j = y + j
            idx = global_i * matrix_size + global_j
            result.append(image_paths[idx])
    return result

def get_expanding_square_order(k, x, y, image_paths):
    matrix_size = 5  # Assumes a 10x10 matrix
    result = []

    for size in range(1, k + 1):
        # Iterate over the new square of size `size x size`
        for i in range(size):
            for j in range(size):
                xi, yj = x + i, y + j
                if xi < matrix_size and yj < matrix_size:
                    index = xi * matrix_size + yj
                    coord = (i, j)
                    # Only include the newly added positions (i == size - 1 or j == size - 1)
                    if i == size - 1 or j == size - 1:
                        result.append(image_paths[index])

    return result

print(diagonal_submatrix(5, 0, 0, paths))
# Expected: ['0000.jpg', '0010.jpg', '0001.jpg', '0020.jpg', '0011.jpg', '0002.jpg', '0021.jpg', '0012.jpg', '0022.jpg']

print(get_expanding_square_order(5, 0, 0, paths))
# Output: ['0000.jpg', '0010.jpg', '0001.jpg', '0011.jpg', '0020.jpg', '0021.jpg', '0002.jpg', '0012.jpg', '0022.jpg']

['0000.jpg', '0005.jpg', '0001.jpg', '0010.jpg', '0006.jpg', '0002.jpg', '0015.jpg', '0011.jpg', '0007.jpg', '0003.jpg', '0020.jpg', '0016.jpg', '0012.jpg', '0008.jpg', '0004.jpg', '0021.jpg', '0017.jpg', '0013.jpg', '0009.jpg', '0022.jpg', '0018.jpg', '0014.jpg', '0023.jpg', '0019.jpg', '0024.jpg']
['0000.jpg', '0001.jpg', '0005.jpg', '0006.jpg', '0002.jpg', '0007.jpg', '0010.jpg', '0011.jpg', '0012.jpg', '0003.jpg', '0008.jpg', '0013.jpg', '0015.jpg', '0016.jpg', '0017.jpg', '0018.jpg', '0004.jpg', '0009.jpg', '0014.jpg', '0019.jpg', '0020.jpg', '0021.jpg', '0022.jpg', '0023.jpg', '0024.jpg']


In [ ]:
matrix_size_wanted = 10
paths = diagonal_submatrix(matrix_size_wanted, 0, 0, paths)
# paths = ['0000.jpg', '0001.jpg', '0002.jpg', '0010.jpg', '0011.jpg', '0012.jpg', '0020.jpg', '0021.jpg', '0022.jpg']
print(paths)

def filter_paths(paths):
  def is_ok(x):
    return int(x[2]) < 5 and int(x[3]) < 5
  return [x for x in paths if is_ok(x)]
paths = filter_paths(paths)
print(paths)

['0000.jpg', '0010.jpg', '0001.jpg', '0020.jpg', '0011.jpg', '0002.jpg', '0030.jpg', '0021.jpg', '0012.jpg', '0003.jpg', '0040.jpg', '0031.jpg', '0022.jpg', '0013.jpg', '0004.jpg', '0050.jpg', '0041.jpg', '0032.jpg', '0023.jpg', '0014.jpg', '0005.jpg', '0060.jpg', '0051.jpg', '0042.jpg', '0033.jpg', '0024.jpg', '0015.jpg', '0006.jpg', '0070.jpg', '0061.jpg', '0052.jpg', '0043.jpg', '0034.jpg', '0025.jpg', '0016.jpg', '0007.jpg', '0080.jpg', '0071.jpg', '0062.jpg', '0053.jpg', '0044.jpg', '0035.jpg', '0026.jpg', '0017.jpg', '0008.jpg', '0090.jpg', '0081.jpg', '0072.jpg', '0063.jpg', '0054.jpg', '0045.jpg', '0036.jpg', '0027.jpg', '0018.jpg', '0009.jpg', '0091.jpg', '0082.jpg', '0073.jpg', '0064.jpg', '0055.jpg', '0046.jpg', '0037.jpg', '0028.jpg', '0019.jpg', '0092.jpg', '0083.jpg', '0074.jpg', '0065.jpg', '0056.jpg', '0047.jpg', '0038.jpg', '0029.jpg', '0093.jpg', '0084.jpg', '0075.jpg', '0066.jpg', '0057.jpg', '0048.jpg', '0039.jpg', '0094.jpg', '0085.jpg', '0076.jpg', '0067.jpg', '00

In [ ]:
len_paths1 = (matrix_size_wanted * (matrix_size_wanted + 1)) // 2
paths1 = paths[:len_paths1]
paths2 = paths[len_paths1:][::-1]

print(paths1)
print(paths2)

['0000.jpg', '0010.jpg', '0001.jpg', '0020.jpg', '0011.jpg', '0002.jpg', '0030.jpg', '0021.jpg', '0012.jpg', '0003.jpg', '0040.jpg', '0031.jpg', '0022.jpg', '0013.jpg', '0004.jpg', '0041.jpg', '0032.jpg', '0023.jpg', '0014.jpg', '0042.jpg', '0033.jpg', '0024.jpg', '0043.jpg', '0034.jpg', '0044.jpg']
[]


In [ ]:
import os

def increment_filename_by_11(filepath):
    # Extract the filename
    # return filepath
    filename = os.path.basename(filepath)

    # Remove the file extension and convert to integer
    number_str = filename[:-4]  # Removes '.jpg'
    number = int(number_str)

    # Add 11
    new_number = number + 21

    # Format back to 4-digit string and add extension
    new_filename = f"{new_number:04}.jpg"
    return new_filename

# Examples
print(increment_filename_by_11("./path_folder/0010.jpg"))  # Output: 0021.jpg
print(increment_filename_by_11("0001.jpg"))                # Output: 0012.jpg


0031.jpg
0022.jpg


In [ ]:
for i in tqdm(range(0, len(paths))):
    print(paths[i - 1], paths[i], sep="\n")
    print(f"0000-{paths[i - 1][:-4]}_LightGlue_1.0.jpg")
    print(i)
    print("-----------------")
    if i == 0:
        continue
    elif i == 1:
        stitched_img_path = LightGlue_matrix_scan_diagonal_matching(
            img_path1 = os.path.join(input_path, increment_filename_by_11(paths[i - 1])),
            img_path2 = os.path.join(input_path, increment_filename_by_11(paths[i])),
            basepath_img2 = paths[i],
            path_list = paths,
            i_value = i,
            matrix_size = 5,
            output_dir=output_path,
            ransac_thresholding=100,
            percentage_of_image_used=1.0
        )
        continue

    stitched_img_path = LightGlue_matrix_scan_diagonal_matching(
        img_path1 = os.path.join(output_path, f"0000-{increment_filename_by_11(paths[i - 1])[:-4]}_LightGlue_1.0.jpg"),
        img_path2 = os.path.join(input_path, increment_filename_by_11(paths[i])),
        basepath_img2 = paths[i],
        path_list = paths,
        i_value = i,
        matrix_size = 5,
        output_dir=output_path,
        ransac_thresholding=100,
        percentage_of_image_used=1.0
    )

  0%|          | 0/25 [00:00<?, ?it/s]

0044.jpg
0000.jpg
0000-0044_LightGlue_1.0.jpg
0
-----------------
0000.jpg
0010.jpg
0000-0000_LightGlue_1.0.jpg
1
-----------------
We want to crop the matrix crop (1, 0) out of a (4, 4) matrix!
1, 1
Is tensor
Image size of (1375, 1843)
Tile size of (1375, 1843)
h_size = 1375
w_size = 1843
i = 0
j = 0
Box proposed is (0, 0, 1375, 1843)



Avem 4261 matches!



Updated maxx matches!
inliers/matches: 4239/4261


  8%|▊         | 2/25 [00:29<05:43, 14.92s/it]

Saving at ./drive/MyDrive/Disertatie/LightGlue/SIFT/051-04-80/0000-0031_LightGlue_1.0.jpg
0010.jpg
0001.jpg
0000-0010_LightGlue_1.0.jpg
2
-----------------
We want to crop the matrix crop (0, 1) out of a (4, 4) matrix!
2, 1
Is tensor
Image size of (1930, 1504)
Tile size of (965, 1504)
h_size = 965
w_size = 1504
i = 0
j = 0
Box proposed is (0, 0, 1592, 1504)



Avem 3743 matches!



Updated maxx matches!
Is tensor
Image size of (1930, 1504)
Tile size of (965, 1504)
h_size = 965
w_size = 1504
i = 1
j = 0
Box proposed is (0, 526, 1592, 1504)
While doing Deep Learning matching got exception: CUDA out of memory. Tried to allocate 1.83 GiB. GPU 0 has a total capacity of 39.56 GiB of which 1.03 GiB is free. Process 2817 has 38.52 GiB memory in use. Of the allocated memory 36.69 GiB is allocated by PyTorch, and 1.34 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See do

 12%|█▏        | 3/25 [00:58<07:37, 20.79s/it]

...

 92%|█████████▏| 23/25 [1:05:01<10:47, 323.95s/it]

0043.jpg
0034.jpg
0000-0043_LightGlue_1.0.jpg
23
-----------------
We want to crop the matrix crop (3, 4) out of a (4, 4) matrix!
5, 5
Is tensor
Image size of (7892, 13040)
Tile size of (1578, 2608)
h_size = 1578
w_size = 2608
i = 0
j = 0
Box proposed is (0, 0, 2603, 4303)



Avem 0 matches!



Is tensor
Image size of (7892, 13040)
Tile size of (1578, 2608)
h_size = 1578
w_size = 2608
i = 0
j = 1
Box proposed is (552, 0, 4181, 4303)



Avem 0 matches!



Is tensor
Image size of (7892, 13040)
Tile size of (1578, 2608)
h_size = 1578
w_size = 2608
i = 0
j = 2
Box proposed is (2130, 0, 5759, 4303)



Avem 0 matches!



Is tensor
Image size of (7892, 13040)
Tile size of (1578, 2608)
h_size = 1578
w_size = 2608
i = 0
j = 3
Box proposed is (3708, 0, 7337, 4303)



Avem 0 matches!



Is tensor
Image size of (7892, 13040)
Tile size of (1578, 2608)
h_size = 1578
w_size = 2608
i = 0
j = 4
Box proposed is (5286, 0, 7892, 4303)



Avem 0 matches!



Is tensor
Image size of (7892, 13040)
Tile size o

 96%|█████████▌| 24/25 [1:11:43<05:47, 347.39s/it]

0034.jpg
0044.jpg
0000-0034_LightGlue_1.0.jpg
24
-----------------
We want to crop the matrix crop (4, 4) out of a (4, 4) matrix!
5, 5
Is tensor
Image size of (7477, 7334)
Tile size of (1495, 1466)
h_size = 1495
w_size = 1466
i = 0
j = 0
Box proposed is (0, 0, 2466, 2418)



Avem 0 matches!



Is tensor
Image size of (7477, 7334)
Tile size of (1495, 1466)
h_size = 1495
w_size = 1466
i = 0
j = 1
Box proposed is (523, 0, 3961, 2418)



Avem 0 matches!



Is tensor
Image size of (7477, 7334)
Tile size of (1495, 1466)
h_size = 1495
w_size = 1466
i = 0
j = 2
Box proposed is (2018, 0, 5456, 2418)



Avem 0 matches!



Is tensor
Image size of (7477, 7334)
Tile size of (1495, 1466)
h_size = 1495
w_size = 1466
i = 0
j = 3
Box proposed is (3513, 0, 6951, 2418)



Avem 0 matches!



Is tensor
Image size of (7477, 7334)
Tile size of (1495, 1466)
h_size = 1495
w_size = 1466
i = 0
j = 4
Box proposed is (5008, 0, 7477, 2418)



Avem 0 matches!



Is tensor
Image size of (7477, 7334)
Tile size of (149

100%|██████████| 25/25 [1:16:07<00:00, 182.69s/it]


In [ ]:
torch.cuda.empty_cache()

In [ ]:
import shutil

folder_path = "./kaze_output"  # Replace with your folder path
zip_path = "kaze_output.zip"

shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)

'/content/kaze_output.zip'

In [ ]:
# shutil.rmtree("./kaze_output")

In [ ]:
import csv

def calculate_averages(csv_filename):
    rmse_total = 0
    psnr_total = 0
    count = 0

    with open(csv_filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if not any(keyword in row['filename'] for keyword in ["83-0084", "96-0097"]):
              rmse_total += float(row['rmse_score'])
              psnr_total += float(row['psnr_score'])
              count += 1

    if count == 0:
        print("No data found in the file.")
        return

    rmse_avg = rmse_total / count
    psnr_avg = psnr_total / count

    print(f"Count: {count}")
    print(f"Average RMSE Score: {rmse_avg}")
    print(f"Average PSNR Score: {psnr_avg}")

In [ ]:
calculate_averages('./kaze_output/kaze.txt')

Count: 88
Average RMSE Score: 49.85837507247925
Average PSNR Score: 14.199706025113324
